In [1]:
import pandas as pd
import numpy as np

## 一、文件的读入和写入

### 1、文件读取

pandas 可以读取的文件格式有很多，这里主要介绍读取 csv, excel, txt 文件。

In [2]:
df_csv = pd.read_csv('./data/ch2/my_csv.csv')
df_csv

df_txt = pd.read_table('./data/ch2/my_table.txt')
df_txt

df_excel = pd.read_excel('./data/ch2/my_excel.xlsx')
df_excel


,col1,col2,col3,col4,col5
0,2,a,1.4,apple,2020/1/1
1,3,b,3.4,banana,2020/1/2
2,6,c,2.5,orange,2020/1/5


常用公共参数：

- `header=None`表示第一行不作为列名
- `index_col`表示把某一列或几列作为索引，见第三章。
- `usecols`表示读取列的集合，**默认读取所有列**。
- `parse_dates`表示需要转换为时间的列，**默认False**。见第十章。
- `nrows`表示读取的数据行数。

In [3]:
pd.read_table('./data/ch2/my_table.txt',header=None)

,0,1,2,3,4
0,col1,col2,col3,col4,col5
1,2,a,1.4,apple,2020/1/1
2,3,b,3.4,banana,2020/1/2
3,6,c,2.5,orange,2020/1/5


In [4]:
pd.read_csv('./data/ch2/my_csv.csv',index_col=['col1','col2'])
pd.read_table('./data/ch2/my_table.txt',index_col=['col1','col2'])
pd.read_csv('./data/ch2/my_csv.csv',parse_dates=['col5'])


,col1,col2,col3,col4,col5
0,2,a,1.4,apple,2020-01-01
1,3,b,3.4,banana,2020-01-02
2,6,c,2.5,orange,2020-01-05


在读取 txt 文件时，经常遇到分隔符非空格的情况， read_table 有一个分割参数 sep ，它使得用户可以自定义分割符号，进行 txt 数据的读取。例如，下面的读取的表以 |||| 为分割：

sep是正则参数，因此需要进行转义，\|，详细见第8章。

In [5]:
pd.read_table('./data/ch2/my_table_special_sep.txt')

,col1 |||| col2
0,TS |||| This is an apple.
1,GQ |||| My name is Bob.
2,WT |||| Well done!


In [6]:
# 上面的结果显然不是理想的，这时可以使用 sep ，同时需要指定引擎为 python
pd.read_table('./data/ch2/my_table_special_sep.txt',sep='\|\|\|\|',engine='python')


,col1,col2
0,TS,This is an apple.
1,GQ,My name is Bob.
2,WT,Well done!


### 2、 数据写入
一般在数据写入中，最常用的操作是把 index 设置为 False ，特别当索引没有特殊意义的时候，这样的行为能把索引在保存的时候去除。

In [7]:
df_csv.to_csv('./data/ch2/my_csv_saved.csv',index=False)
df_excel.to_excel('./data/ch2/my_csv_saved.xlsx',index=False)

pandas没有定义to_table函数，to_csv可以保存txt文件，允许使用自定义分隔符号，常用\t分隔。


In [8]:
df_txt.to_csv('./data/ch2/my_txt_save.txt',sep='\t',index=False)

想要把表格快速转换为markdown和latex，可以使用to_markdown和to_latex函数，但需要安装tabulate包

In [9]:
# pip install tabulate

print(df_csv.to_markdown())

|    |   col1 | col2   |   col3 | col4   | col5     |
|---:|-------:|:-------|-------:|:-------|:---------|
|  0 |      2 | a      |    1.4 | apple  | 2020/1/1 |
|  1 |      3 | b      |    3.4 | banana | 2020/1/2 |
|  2 |      6 | c      |    2.5 | orange | 2020/1/5 |


In [10]:
print(df_csv.to_latex())

\begin{tabular}{lrlrll}
\toprule
 & col1 & col2 & col3 & col4 & col5 \\
\midrule
0 & 2 & a & 1.400000 & apple & 2020/1/1 \\
1 & 3 & b & 3.400000 & banana & 2020/1/2 \\
2 & 6 & c & 2.500000 & orange & 2020/1/5 \\
\bottomrule
\end{tabular}



## 二、基本数据结构

pandas 中具有两种基本的数据存储结构，存储一维 values 的 Series 和存储二维 values 的 DataFrame ，在这两种结构上定义了很多的属性和方法。

### 1. Series
Series 一般由四个部分组成，分别是序列的值 data 、索引 index 、存储类型 dtype 、序列的名字 name 。其中，索引也可以指定它的名字，默认为空。

In [11]:
s = pd.Series(data = [100,'a',{'dict':5}],
              index = pd.Index(['id',20,'third'],name='my_index'),
              dtype='object',
              name='my_name'
              )
s

my_index
id               100
20                 a
third    {'dict': 5}
Name: my_name, dtype: object

**object类型**

object 代表了一种混合类型，正如上面的例子中存储了整数、字符串以及 Python 的字典数据结构。此外，目前 pandas 把纯字符串序列也默认认为是一种 object 类型的序列，但它也可以用 string 类型存储，文本序列的内容会在第八章中讨论。

In [12]:
# 这些属性，可以通过.来获取
s.values
# array([100, 'a', {'dict': 5}], dtype=object)

s.index
# Index(['id', 20, 'third'], dtype='object', name='my_index')

s.dtype
# dtype('O')

s.name
# 'my_name'

# .shape可以获取序列的长度
s.shape
# (3,)

# s[idx]可以通过索引获取值
s['id']
# 100


100

### 2.DataFrame

DataFrame在Series的基础上增加了列索引，一个数据框可以由二维的data与列索引来构造。

In [13]:
data = [[1,'a',1.2],[2, 'b', 2.2],[3, 'c', 3.2]]
df = pd.DataFrame(data = data,
                  index = ['row_%d'%i for i in range(3)],
                  columns=['col_0','col_1','col_2'])
df

,col_0,col_1,col_2
row_0,1,a,1.2
row_1,2,b,2.2
row_2,3,c,3.2


但一般而言，会采用从列索引名到数据的映射来构造数据框，同时再加上行索引。

In [14]:
df = pd.DataFrame(data = {'col0':[1,2,3],'col1':list('abc'),'col2':[1.2,2.2,3.2]},
                  index = ['row_%d'%i for i in range(3)])
df

,col0,col1,col2
row_0,1,a,1.2
row_1,2,b,2.2
row_2,3,c,3.2


由于这种映射关系，在DataFrame中可以用[col_name]与[col_list]来取出来对应的列和多个列组成的表，结果分别是Series和DataFrame:

In [15]:
df['col0']
# row_0    1
# row_1    2
# row_2    3
# Name: col0, dtype: int64

df[['col0','col1']]

,col0,col1
row_0,1,a
row_1,2,b
row_2,3,c


In [16]:
# 与Series类似，在数据框里面同样可以取出相应的属性：

df.values
# array([[1, 'a', 1.2],
    #    [2, 'b', 2.2],
    #    [3, 'c', 3.2]], dtype=object)

df.index
# Index(['row_0', 'row_1', 'row_2'], dtype='object')

df.columns
# Index(['col0', 'col1', 'col2'], dtype='object')

# 返回的是值为相应列数据类型的Series
df.dtypes 
# col0      int64
# col1     object
# col2    float64
# dtype: object

df.shape
# (3, 3)

(3, 3)

可以通过.T把DataFrame进行转置

In [17]:
df.T

,row_0,row_1,row_2
col0,1,2,3
col1,a,b,c
col2,1.2,2.2,3.2


## 三、常用基本函数

为了进行举例说明，在接下来的部分和其余章节都将会使用一份 learn_pandas.csv 的虚拟数据集，它记录了四所学校学生的体测个人信息。

In [18]:
df = pd.read_csv('./data/learn_pandas.csv')
df.head()

df.columns
# Index(['School', 'Grade', 'Name', 'Gender', 'Height', 'Weight', 'Transfer',
    #    'Test_Number', 'Test_Date', 'Time_Record'],
    #   dtype='object')

df.dtypes
#School          object
# Grade           object
# Name            object
# Gender          object
# Height         float64
# Weight         float64
# Transfer        object
# Test_Number      int64
# Test_Date       object
# Time_Record     object
# dtype: object


School          object
Grade           object
Name            object
Gender          object
Height         float64
Weight         float64
Transfer        object
Test_Number      int64
Test_Date       object
Time_Record     object
dtype: object

上述列名依次代表学校、年级、姓名、性别、身高、体重、是否为转系生、体测场次、测试时间、1000米成绩，本章只需使用其中的前七列。

In [19]:
df = df[df.columns[:7]]

### 1.汇总函数

- head,tail函数分别表示返回表或者序列的前n行和后n行，默认n=5
- info,describe函数分别返回表的 **信息概况**和表中**数值列对应的主要统计量**

info,describe函数只能实现较少信息的展示，如果需要对一份数据进行全面且有效的观察，特别是在列比较多的情况下，推荐使用[pandas-profiling](https://pandas-profiling.github.io/pandas-profiling/docs/master/index.html)包。

In [20]:
df.head(2)

df.tail(3)

,School,Grade,Name,Gender,Height,Weight,Transfer
197,A,Senior,Chengqiang Chu,Female,153.9,45.0,N
198,A,Senior,Chengmei Shen,Male,175.3,71.0,N
199,D,Sophomore,Chunpeng Lv,Male,155.7,51.0,N


In [21]:
# 信息概况
df.info()
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 200 entries, 0 to 199
# Data columns (total 7 columns):
#    Column    Non-Null Count  Dtype  
# ---  ------    --------------  -----  
#  0   School    200 non-null    object 
#  1   Grade     200 non-null    object 
#  2   Name      200 non-null    object 
#  3   Gender    200 non-null    object 
#  4   Height    183 non-null    float64
#  5   Weight    189 non-null    float64
#  6   Transfer  188 non-null    object 
# dtypes: float64(2), object(5)
# memory usage: 11.1+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   School    200 non-null    object 
 1   Grade     200 non-null    object 
 2   Name      200 non-null    object 
 3   Gender    200 non-null    object 
 4   Height    183 non-null    float64
 5   Weight    189 non-null    float64
 6   Transfer  188 non-null    object 
dtypes: float64(2), object(5)
memory usage: 11.1+ KB


In [22]:
# 数值列的主要统计量
df.describe()

,Height,Weight
count,183.000000,189.000000
mean,163.218033,55.015873
std,8.608879,12.824294
min,145.400000,34.000000
25%,157.150000,46.000000
50%,161.900000,51.000000
75%,167.500000,65.000000
max,193.900000,89.000000


### 2.特征统计函数

在Series和DataFrame上定义了许多统计函数，最常见的是sum,mean,median,var,std,max,min等。例如，选出身高和体重列进行演示：

In [23]:
df_demo = df[['Weight','Height']]

df_demo.mean()
# Weight     55.015873
# Height    163.218033
# dtype: float64

# 中位数 
df_demo.median()

# 
df_demo.sum()
# Weight    10398.0
# Height    29868.9
# dtype: float64

Weight    10398.0
Height    29868.9
dtype: float64

count,quantile,idxmax函数：
- count函数返回非缺失值的数量
- quantile函数返回分位数
- idxmax函数返回最大值的索引,idxmin函数返回最小值的索引

In [24]:
# 分位数
df_demo.quantile(0.75)
# Weight     65.0
# Height    167.5
# Name: 0.75, dtype: float64

# 非缺失值的数量
df_demo.count()
# Weight    189
# Height    183
# dtype: int64

# 最大值的索引
df_demo.idxmax()
# Weight      2
# Height    193
# dtype: int64

Weight      2
Height    193
dtype: int64

上面的所有函数都有axis参数,用于指定计算的轴,默认是0,表示列聚合，1表示按行聚合。

In [25]:
df_demo.mean(axis=1).head()

0    102.45
1    118.25
2    138.95
3     41.00
4    124.00
dtype: float64

### 3.唯一值函数
- unique函数返回唯一值的数组
- nunique函数返回唯一值的数量
- value_counts函数可以得到唯一值和其对应的频数

In [26]:
df['School'].unique()
# array(['A', 'B', 'C', 'D'], dtype=object)

df['School'].nunique()
# 4

4

In [27]:
df['School'].value_counts()
# School
# D    69
# A    57
# C    40
# B    34
# Name: count, dtype: int64

School
D    69
A    57
C    40
B    34
Name: count, dtype: int64

如果要观察多列组合的唯一值，可以使用**drop_duplicates**函数，关键参数是keep,默认是'first',表示保留第一次出现的组合,如果是'last',表示保留最后一次出现的组合,如果是False,表示删除所有重复的组合。

此外,duplicated函数可以用于判断每一行是否是重复的,返回一个布尔值的Series；keep参数和上面的一致，返回序列会把重复元素设为True,否则为False.

drop_duplicates相当于把duplicated为True的对应行剔除。

In [ ]:
df_demo = df[['Gender', 'Transfer','Name']]

# 对Transfer和Gender列去重，不含其他列
df_demo[['Gender', 'Transfer']].drop_duplicates()
# 对Transfer和Gender列去重，包含其他列
df_demo.drop_duplicates(['Gender', 'Transfer'])


# Gender	Transfer	Name
# 0	Female	N	Gaopeng Yang
# 1	Male	N	Changqiang You
# 12	Female	NaN	Peng You
# 21	Male	NaN	Xiaopeng Shen
# 36	Male	Y	Xiaojuan Qin
# 43	Female	Y	Gaoli Feng

df_demo.drop_duplicates(['Gender','Transfer'],keep='last')
df_demo = df[['Gender', 'Transfer','Name']]

df_demo.drop_duplicates(['Gender', 'Transfer'])

# Gender	Transfer	Name
# 0	Female	N	Gaopeng Yang
# 1	Male	N	Changqiang You
# 12	Female	NaN	Peng You
# 21	Male	NaN	Xiaopeng Shen
# 36	Male	Y	Xiaojuan Qin
# 43	Female	Y	Gaoli Feng

df_demo.drop_duplicates(['Gender','Transfer'],keep='last')
# Gender	Transfer	Name
# 147	Male	NaN	Juan You
# 150	Male	Y	Chengpeng You
# 169	Female	Y	Chengquan Qin
# 194	Female	NaN	Yanmei Qian
# 197	Female	N	Chengqiang Chu
# 199	Male	N	Chunpeng Lv

# # 保留只出现过一次的性别和姓名组合
df_demo.drop_duplicates(['Gender','Transfer'],keep=False)

df['School'].drop_duplicates()
# 0    A
# 1    B
# 3    C
# 5    D
# Name: School, dtype: object

0    A
1    B
3    C
5    D
Name: School, dtype: object

In [29]:
df_demo.duplicated(['Gender','Transfer']).head()
# df_demo.head()
# 0    False
# 1    False
# 2     True
# 3     True
# 4     True
# dtype: bool

df['School'].duplicated().head() # 在Series上也可以使用
# 0    False
# 1    False
# 2     True
# 3    False
# 4     True
# Name: School, dtype: bool

0    False
1    False
2     True
3    False
4     True
Name: School, dtype: bool

### 4.替换函数

一般而言，**替换操作是针对某一个列进行的**。
因此下面的例子都以 Series 举例。 pandas 中的替换函数可以归纳为三类：**映射替换、逻辑替换、数值替换**。其中映射替换包含 replace 方法、第八章中的 str.replace 方法以及第九章中的 cat.codes 方法，此处介绍 replace 的用法。

在 replace 中，可以通过字典构造，或者传入两个列表来进行替换：

In [30]:
# 传入字典进行替换
df['Gender'].replace({'Female':0,'Male':1}).head()


df['Gender'].replace(['Female','Male'],[0,1]).head()
# 0    0
# 1    1
# 2    1
# 3    0
# 4    1
# Name: Gender, dtype: int64

C:\Users\ZJJ\AppData\Local\Temp\ipykernel_14456\770359339.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Gender'].replace({'Female':0,'Male':1}).head()
C:\Users\ZJJ\AppData\Local\Temp\ipykernel_14456\770359339.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Gender'].replace(['Female','Male'],[0,1]).head()


0    0
1    1
2    1
3    0
4    1
Name: Gender, dtype: int64

另外， replace 还有一种特殊的方向替换,指定method参数:
- ffill，用前一个最近的未被替换的值进行替换。
- bfill，使用后面最近的未被替换的值进行替换。

In [31]:
s = pd.Series(['a',1,'b',2,1,1,'a'])

# ffill使用前一个最近的未被替换的值来替换。
s.replace([1,2],method='ffill').head()
# 0    a
# 1    a
# 2    b
# 3    b
# 4    b
# dtype: object

s.replace([1,2],method='bfill').head()
# 0    a
# 1    b
# 2    b
# 3    a
# 4    a
# dtype: object

C:\Users\ZJJ\AppData\Local\Temp\ipykernel_14456\4104119037.py:4: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  s.replace([1,2],method='ffill').head()
C:\Users\ZJJ\AppData\Local\Temp\ipykernel_14456\4104119037.py:12: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  s.replace([1,2],method='bfill').head()


0    a
1    b
2    b
3    a
4    a
dtype: object

**逻辑替换**包括了 where 和 mask ，这两个函数是完全对称的： where 函数在传入条件为 **False** 的对应行进行替换，而 mask 在传入条件为 True 的对应行进行替换，当不指定替换值时，替换为缺失值。

In [32]:
s = pd.Series([-1,1.235,100,-50])

s.where(s<0)
# 0    -1.0
# 1     NaN
# 2     NaN
# 3   -50.0
# dtype: float64

# 不小于0的，使用100替换
s.where(s<0,100)
# 0     -1.0
# 1    100.0
# 2    100.0
# 3    -50.0
# dtype: float64

# 盖住s<0的部分
s.mask(s<0)
# 0        NaN
# 1      1.235
# 2    100.000
# 3        NaN
# dtype: float64

# 盖住s<0的部分，使用100替换

s.mask(s<0,100)
# 0    100.000
# 1      1.235
# 2    100.000
# 3    100.000
# dtype: float64

0    100.000
1      1.235
2    100.000
3    100.000
dtype: float64

需要注意的是，传入的条件只需是与**被调用的 Series 索引一致的布尔序列**即可：

In [33]:
# 同索引的布尔序列
s_condition = pd.Series([True,False,False,True],index=s.index)
s.mask(s_condition)

0        NaN
1      1.235
2    100.000
3        NaN
dtype: float64

**数值替换**包含了 round, abs, clip 方法，它们分别表示按照给定精度四舍五入、取绝对值和截断：

In [34]:
s = pd.Series([-1,1.23,100,-20])

# 四舍五入
s.round(2)

# 绝对值
s.abs()


# 截断
# clip 前两个数分别表示上下截断边界
s.clip(0, 2)
# 0    0.00
# 1    1.23
# 2    2.00
# 3    0.00
# dtype: float64

0    0.00
1    1.23
2    2.00
3    0.00
dtype: float64

#### 练习：
在 clip 中，超过边界的只能截断为边界值，如果要把超出边界的替换为自定义的值，应当如何做？

In [35]:
# 法一：布尔索引
s = pd.Series([1, 3, 5, 7, 9, 11])
lower = 3  # 下界
upper = 9  # 上界
custom_low = -1  # 低于下界时的替换值
custom_high = 100  # 高于上界时的替换值

# 替换低于下界的值
s[s < lower] = custom_low
# 替换高于上界的值
s[s > upper] = custom_high
s

# 法二：使用where()方法
# where方法的替换是替换不符合条件的元素
s = pd.Series([1, 3, 5, 7, 9, 11])
s = s.where(s >= lower, custom_low)
s = s.where(s <= upper, custom_high)
s

# 法三：使用mask()方法
# mask方法的替换是替换符合条件的元素
s = pd.Series([1, 3, 5, 7, 9, 11])
s = s.mask(s < lower,custom_low)
s = s.mask(s > upper,custom_high)
s

# 0     -1
# 1      3
# 2      5
# 3      7
# 4      9
# 5    100
# dtype: int64

0     -1
1      3
2      5
3      7
4      9
5    100
dtype: int64

## 5.排序函数
排序共有两种方式，其一为值排序，其二为索引排序.
- sort_values() ：值排序
- sort_index() ：索引排序

为了演示排序函数，下面先利用 set_index 方法把年级和姓名两列作为索引，多级索引的内容和索引设置的方法将在第三章进行详细讲解。

In [36]:
df_demo = df[['Grade','Name','Height','Weight']].set_index(['Grade','Name'])

df_demo.head()
# 		Height	Weight
# Grade	Name		
# Freshman	Gaopeng Yang	158.9	46.0
#           Changqiang You	166.5	70.0
# Senior	Mei Sun	188.9	89.0
# Sophomore	Xiaojuan Sun	NaN	41.0
# Gaojuan You	174.0	74.0


Height  Weight
Grade     Name                          
Freshman  Gaopeng Yang     158.9    46.0
          Changqiang You   166.5    70.0
Senior    Mei Sun          188.9    89.0
Sophomore Xiaojuan Sun       NaN    41.0
          Gaojuan You      174.0    74.0

对身高进行排序，默认参数ascending=True为升序：

In [37]:
df_demo.sort_values('Height').head()

df_demo.sort_values('Weight',ascending=False).head()

# Height	Weight
# Grade	Name		
# Senior	Mei Sun	188.9	89.0
        # Qiang Zheng	183.9	87.0
# Freshman	Qiang Han	185.3	87.0
# Senior	Gaoli Zhao	186.5	83.0
# Freshman	Chunli Zhao	180.2	83.0


Height  Weight
Grade    Name                       
Senior   Mei Sun       188.9    89.0
         Qiang Zheng   183.9    87.0
Freshman Qiang Han     185.3    87.0
Senior   Gaoli Zhao    186.5    83.0
Freshman Chunli Zhao   180.2    83.0

排序中，经常用到多列排序的问题，比如在体重相同的情况下，对身高进行排序，并且保持身高排序降序，体重升序：

In [38]:
df_demo.sort_values(['Weight','Height'],ascending=[True,False]).head()
# 		            Height	Weight
# Grade	Name		
# Sophomore	Peng Han	147.8	34.0
# Senior	Gaomei Lv	147.3	34.0
# Junior	Xiaoli Chu	145.4	34.0
# Sophomore	Qiang Zhou	150.5	36.0
# Freshman	Yanqiang Xu	152.4	38.0


,,Height,Weight
Grade,Name,,
Sophomore,Peng Han,147.8,34.0
Senior,Gaomei Lv,147.3,34.0
Junior,Xiaoli Chu,145.4,34.0
Sophomore,Qiang Zhou,150.5,36.0
Freshman,Yanqiang Xu,152.4,38.0


索引排序的用法和值排序完全一致，只不过元素的值在索引中，**此时需要指定索引层的名字或者层号，用参数 level 表示**。另外，需要注意的是**索引排序时，字符串的排列顺序由字母顺序决定**。

In [39]:
df_demo.sort_index(level=['Grade','Name'],ascending=[True,False]).head()
# 		Height	Weight
# Grade	Name		
# Freshman	Yanquan Wang	163.5	55.0
# Yanqiang Xu	152.4	38.0
# Yanqiang Feng	162.3	51.0
# Yanpeng Lv	NaN	65.0
# Yanli Zhang	165.1	52.0


Height  Weight
Grade    Name                         
Freshman Yanquan Wang    163.5    55.0
         Yanqiang Xu     152.4    38.0
         Yanqiang Feng   162.3    51.0
         Yanpeng Lv        NaN    65.0
         Yanli Zhang     165.1    52.0

### 6.apply方法

apply 方法常**用于 DataFrame 的行迭代或者列迭代**,默认是列迭代，它的 axis 含义与第2小节中的统计聚合函数一致， **apply 的参数往往是一个以序列为输入的函数。例如对于 .mean()** ，使用 apply 可以如下地写出：

In [40]:
df_demo = df[['Height','Weight']]
def my_mean(x):
    res = x.mean()
    return res

df_demo.apply(my_mean)
# Height    163.218033
# Weight     55.015873
# dtype: float64

Height    163.218033
Weight     55.015873
dtype: float64

同样的，可以利用 lambda 表达式使得书写简洁，这里的 x 就指代被调用的 df_demo 表中逐个输入的序列：

In [41]:
df_demo.apply(lambda x:x.mean())
# Height    163.218033
# Weight     55.015873
# dtype: float64

Height    163.218033
Weight     55.015873
dtype: float64

若指定 axis=1 ，那么每次传入函数的就是行元素组成的 Series ，其结果与之前的逐行均值结果一致。

In [42]:
df_demo.apply(lambda x:x.mean(),axis=1).head()
# 0    102.45
# 1    118.25
# 2    138.95
# 3     41.00
# 4    124.00
# dtype: float64

0    102.45
1    118.25
2    138.95
3     41.00
4    124.00
dtype: float64

这里再举一个例子： **mad 函数返回的是一个序列中偏离该序列均值的绝对值大小的均值**，例如序列1,3,7,10中，均值为5.25，每一个元素偏离的绝对值为4.25,2.25,1.75,4.75，这个偏离序列的均值为3.25。现在利用 apply 计算升高和体重的 mad 指标：

In [43]:
df_demo.apply(lambda x:(x-x.mean()).abs().mean())

Height     6.707229
Weight    10.391870
dtype: float64

In [44]:
# 新版本已经丢弃了mad函数
# df_demo.mad()

得益于传入自定义函数的处理， **apply 的自由度很高，但这是以性能为代价的**。一般而言，使用 pandas 的内置函数处理和 apply 来处理同一个任务，其速度会相差较多，因此只有在确实存在自定义需求的情境下才考虑使用 apply 。

## 四、窗口对象
pandas 中有3类窗口，分别是**滑动窗口 rolling 、扩张窗口 expanding 以及指数加权窗口 ewm** 。需要说明的是，以日期偏置为窗口大小的滑动窗口将在第十章讨论，指数加权窗口见本章练习。

### 1. 滑窗对象
要使用滑窗函数，就必须先要对一个序列使用 .rolling 得到滑窗对象，其最重要的参数为窗口大小 window 。

In [45]:
s = pd.Series([1,2,3,4,5])

roller = s.rolling(window=3)
roller
# Rolling [window=3,center=False,axis=0,method=single]

Rolling [window=3,center=False,axis=0,method=single]

有了滑窗对象，就可以对其使用各种聚合函数了，例如 mean, sum, median, std 等。
需要注意的是窗口包含当前行所在的元素，例如在第四个位置进行均值运算时，应当计算(2+3+4)/3，而不是(1+2+3)/3：

In [46]:
roller.mean()
# 0    NaN
# 1    NaN
# 2    2.0
# 3    3.0
# 4    4.0
# dtype: float64

roller.sum()
# 0     NaN
# 1     NaN
# 2     6.0
# 3     9.0
# 4    12.0
# dtype: float64

0     NaN
1     NaN
2     6.0
3     9.0
4    12.0
dtype: float64

滑动相关系数或协方差

In [47]:
s2 = pd.Series([1,2,6,16,30])

# 计算协方差
roller.cov(s2)
# 0     NaN
# 1     NaN
# 2     2.5
# 3     7.0
# 4    12.0
# dtype: float64

# 计算相关系数
roller.corr(s2)
# 0         NaN
# 1         NaN
# 2    0.944911
# 3    0.970725
# 4    0.995402
# dtype: float64

0         NaN
1         NaN
2    0.944911
3    0.970725
4    0.995402
dtype: float64

此外，还支持使用 apply 传入自定义函数，其传入值是对应窗口的 Series ，例如上述的均值函数可以等效表示：

shift,diff,pct_change是一组类滑窗函数，公共参数periods=n，默认为1，
- shift表示取前n个元素的值，
- diff与向前第n个元素做差（与numpy不同，后者表示n阶差分）、
- pct_change与向前第n个元素相比计算增长率。
其中n表示向前第n个元素，若n为负数则表示向后第n个元素。

a b   增长率 (b-a)/a
0 1


In [48]:
s = pd.Series([1,3,6,10,15])
# 取前面第2个数
s.shift(2)
# 解释：0位置没有数，所以是NaN
# 1位置没有数，所以是NaN
# 2位置的数是1，所以是1
# 3位置的数是3，所以是3
# 4位置的数是6，所以是6

# 0    NaN
# 1    NaN
# 2    1.0
# 3    3.0
# 4    6.0
# dtype: float64

# 计算当前数与前面数的差值
s.diff()
# 0    NaN
# 1    2.0
# 2    3.0
# 3    4.0
# 4    5.0
# dtype: float64

s.diff(2)
# 0    NaN
# 1    NaN
# 2    5.0
# 3    7.0
# 4    9.0
# dtype: float64

# 与向前第n个元素相比计算增长率。
s.pct_change()
# 0         NaN
# 1    2.000000
# 2    1.000000
# 3    0.666667
# 4    0.500000
# dtype: float64

# 取后面第1个数
s.shift(-1)
# 0     3.0
# 1     6.0
# 2    10.0
# 3    15.0
# 4     NaN
# dtype: float64

0     3.0
1     6.0
2    10.0
3    15.0
4     NaN
dtype: float64

将其视作类滑窗函数的原因是，它们的**功能可以用窗口大小为 n+1 的 rolling 方法等价代替**：

In [49]:
s.rolling(3).apply(lambda x:list(x)[0]) # s.shift(2)
# 0    NaN
# 1    NaN
# 2    1.0
# 3    3.0
# 4    6.0
# dtype: float64

# s.diff(3)
s.rolling(4).apply(lambda x:list(x)[-1] - list(x)[0])
# 0     NaN
# 1     NaN
# 2     NaN
# 3     9.0
# 4    12.0
# dtype: float64

# s.pct_change(3)
def my_pct(x):
    L = list(x)
    return L[-1]/L[0]-1
s.rolling(4).apply(my_pct)
# 0    NaN
# 1    NaN
# 2    NaN
# 3    9.0
# 4    4.0
# dtype: float64

0    NaN
1    NaN
2    NaN
3    9.0
4    4.0
dtype: float64

#### 练习：

rolling 对象的默认窗口方向都是向前的，某些情况下用户需要向后的窗口，例如对1,2,3设定向后窗口为2的 sum 操作，结果为3,5,NaN，此时应该如何实现向后的滑窗操作？

In [50]:
a = pd.Series([1,2,3])
# 先翻转
a = a[::-1]
# 再计算
a = a.rolling(2).sum()
# 最后再翻转回来
a = a[::-1]
a
# 0    3.0
# 1    5.0
# 2    NaN
# dtype: float64

0    3.0
1    5.0
2    NaN
dtype: float64

### 2.扩张窗口
扩张窗口又称累计窗口，可以理解为一个动态长度的窗口，其窗口的大小就是从序列开始处到具体操作的对应位置，其使用的聚合函数会作用于这些逐步扩张的窗口上。具体地说，设序列为a1, a2, a3, a4，则其每个位置对应的窗口即[a1]、[a1, a2]、[a1, a2, a3]、[a1, a2, a3, a4]。

In [51]:
s = pd.Series([1,3,6,10])
s.expanding().mean()
# 0    1.000000
# 1    2.000000
# 2    3.333333
# 3    5.000000
# dtype: float64

0    1.000000
1    2.000000
2    3.333333
3    5.000000
dtype: float64

#### 练习：

cummax, cumsum, cumprod 函数是典型的类扩张窗口函数，请使用 expanding 对象依次实现它们。

In [52]:
s.expanding().max()

s.expanding().sum()

s.expanding().apply(np.prod)
# 0      1.0
# 1      3.0
# 2     18.0
# 3    180.0
# dtype: float64
s.cumprod()
# 0      1
# 1      3
# 2     18
# 3    180
# dtype: int64


0      1
1      3
2     18
3    180
dtype: int64

## 五、练习,
### Ex1：口袋妖怪数据集,
现有一份口袋妖怪的数据集，下面进行一些背景说明：

- `#` 代表全国图鉴编号，不同行存在相同数字则表示为该妖怪的不同状态

- 妖怪具有单属性和双属性两种，对于单属性的妖怪， Type 2 为缺失值

- Total, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed 分别代表种族值、体力、物攻、防御、特攻、特防、速度，其中种族值为后6项之和

1. 对 HP, Attack, Defense, Sp. Atk, Sp. Def, Speed 进行加总，验证是否为 Total 值。

2. 对于 # 重复的妖怪只保留第一条记录，解决以下问题：

a.求第一属性的种类数量和前三多数量对应的种类

b.求第一属性和第二属性的组合种类

c.求尚未出现过的属性组合

3. 按照下述要求，构造 Series ：

a.取出物攻，超过120的替换为 high ，不足50的替换为 low ，否则设为 mid

b.取出第一属性，分别用 replace 和 apply 替换所有字母为大写

c.求每个妖怪六项能力的离差，即所有能力中偏离中位数最大的值，添加到 df 并从大到小排序


In [53]:
df = pd.read_csv('./data/ch2/pokemon.csv')

df.head(30)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


#### Ex1答案

In [54]:
# 1. 对 HP, Attack, Defense, Sp. Atk, Sp. Def, Speed 进行加总，验证是否为 Total 值。
df_t1 = df[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']]
A = df['Total'] == df_t1.sum(axis=1)
# A.count()
A.all()
# np.True_

# 2. 对于 # 重复的妖怪只保留第一条记录，解决以下问题：
df_t2 = df.drop_duplicates('#',keep='first')
# 查看#不重复的记录有多少条
print(df_t2['#'].nunique())
print(df_t2['#'].count())
# 721
    # a.求第一属性的种类数量和前三多数量对应的种类

df_t2_a = df_t2['Type 1'].nunique()
# df_t2_a
# 18

# value_counts() 得到唯一值和对应频数
df_t2['Type 1'].value_counts()[:3]
# Type 1
# Water     105
# Normal     93
# Grass      66
# Name: count, dtype: int64

    # b.求第一属性和第二属性的组合种类

df_zuhe= df_t2[['Type 1','Type 2']].drop_duplicates(['Type 1','Type 2']).count()

# Type 1    143
# Type 2    125
# dtype: int64

print(df_zuhe.max())
# 143

    # c.求尚未出现过的属性组合
attr1 = df_t2['Type 1'].unique()
attr2 = df_t2['Type 2'].unique()

# 查看两种属性的差集
print(set(attr1).difference(set(attr2)))
# 结果为空集，
# 说明没有属性组合是只出现在Type 1而不出现在Type 2中的

# numpy 排序，默认升序
attr1 = np.sort(attr1)

# 没有对L_full去重！！！
# L_full = [i+' '+j if i!=j else i for i in attr1 for j in attr1 ]
L_full = []
for i in attr1:
    for j in attr1:
        if i==j:
            continue
        if i<j:
            L_full.append(i+' '+j)
        else:
            L_full.append(j+' '+i)

# # 1. 生成实际出现的属性组合（按字符串排序后合并）
# def get_sorted_comb(t1, t2):
#     """
#     对 t1 和 t2 按字符串排序后合并为组合
#     规则：
#     - 若 t2 为空（NaN），则组合为 t1（单属性）
#     - 若 t2 非空，将 [t1, t2] 按字符串排序后拼接
#     """
#     if pd.isna(t2):
#         return t1  # 单属性组合
#     else:
#         sorted_types = sorted([t1, t2])  # 按字母顺序排序字符串
#         return f"{sorted_types[0]} {sorted_types[1]}"

# # 提取实际出现的所有组合（去重）
# L_part = []
# for _, row in df_t2.iterrows():
#     comb = get_sorted_comb(row['Type 1'], row['Type 2'])
#     L_part.append(comb)
# L_part = set(L_part) # 去重


# 我的
# # 需要对Type 1和Type 2按照索引排序然后合并
# # 找出有空值的Type 2序列

# df_T2_notna_T2 = df_t2['Type 2'].loc[df_t2['Type 2'].notna()]
# 下面这么写更规范！
df_T2_isna = df_t2.loc[df_t2['Type 2'].isna(),'Type 1']
L_part1 = list(df_T2_isna)
df_T2_notna_T1 = df_t2.loc[df_t2['Type 2'].notna(),'Type 1']
df_T2_notna_T2 = df_t2.loc[df_t2['Type 2'].notna(),'Type 2']

# 合并
df_T12 = pd.concat([df_T2_notna_T1,df_T2_notna_T2],axis=1)
df_T12.columns = ['Type 1','Type 2']

# GPT的排序和合并
# 定义函数：对单行的Type 1和Type 2排序后返回组合
def sorted_comb(row):
    t1 = row['Type 1']
    t2 = row['Type 2']
    if pd.isna(t2):
        return t1  # 单属性组合
    # 对两个属性排序后拼接
    sorted_types = sorted([t1, t2])
    # print(f"{sorted_types[0]} {sorted_types[1]}")
    return f"{sorted_types[0]} {sorted_types[1]}"

# 直接生成所有实际组合（含单属性和排序后的双属性）
L_part = df_t2.apply(sorted_comb, axis=1).unique().tolist()

# 我的去重和排序
# # 对合并后的Type 1和Type 2进行排序
# df_T12[['Type 1','Type 2']].values.sort(axis=1)
# print(df_T12)
# df_T12['Type 1'] = df_T12[['Type 1','Type 2']].values[:,0]
# df_T12['Type 2'] = df_T12[['Type 1','Type 2']].values[:,1]


L_part = [i+' '+j  for i,j in zip(df_T12['Type 1'],df_T12['Type 2']) ] + L_part1

res = set(L_full).difference(set(L_part))



print(len(res))
with open('./data/ch2/pokemon_type_combs_doubao.txt','w') as f:
    for i in set(res):
        f.write(i+'\n')
# print(type(res))

res_list = list(res)
# 存储已检查过的组合，避免重复判断
checked = set()
# 存储找到的逆序对
reverse_pairs = []

for comb in res_list:
    if comb in checked:
        continue
    # 拆分组合为两个属性（仅针对双属性组合，单属性组合无逆序）
    parts = comb.split(' ')
    if len(parts) != 2:  # 单属性组合（如 "Fire"），无逆序，跳过
        checked.add(comb)
        continue
    # 生成逆序组合（a b → b a）
    reverse_comb = f"{parts[1]} {parts[0]}"
    # 检查逆序组合是否也在 res 中
    if reverse_comb in res_list:
        reverse_pairs.append((comb, reverse_comb))
        checked.add(comb)
        checked.add(reverse_comb)
    else:
        checked.add(comb)

# 输出结果
if reverse_pairs:
    print("res 中存在以下 a b 和 b a 顺序的组合共有{}条：".format(len(reverse_pairs)))
    for pair in reverse_pairs:
        print(f"{pair[0]} 和 {pair[1]}")
else:
    print("res 中不存在 a b 和 b a 顺序的组合")


721
721
143
set()
95
res 中不存在 a b 和 b a 顺序的组合


In [55]:

# 3. 按照下述要求，构造 Series ：

# a.取出物攻，超过120的替换为 high ，不足50的替换为 low ，否则设为 mid

# 我的方法：将数据替换为数值，但不符合题意，题目要求字符串
low,mid,high = 0,80,120
attack = df_t1['Attack'].copy()
attack = attack.mask(attack >= 120,high)
attack = attack.mask(attack < 50,low)

# and 用于 Python 标量，& 用于向量元素级判断
attack.mask((attack >= 50 ) & (attack <120),mid)

# 答案方法：
# 这里一直mask的都是df['Attack'],
# 如果像上面的那种方法一样，逐步去mask然后替换为字符串就会报错。
print(df['Attack'].mask(df['Attack'] > 120,'high')\
    .mask(df['Attack'] < 50,'low')\
    .mask((df['Attack'] >= 50) & (df['Attack'] <= 120),'mid')\
    .head())
# 0    low
# 1    mid
# 2    mid
# 3    mid
# 4    mid
# Name: Attack, dtype: object

# b.取出第一属性，分别用 replace 和 apply 替换所有字母为大写

# replace方法
# 把所有的值和对应的结果替换为字典就可以进行replace了
df['Type 1'].replace({i:i.upper() for i in df['Type 1'].unique()}).head()

# apply方法
df_type1 = df['Type 1'].copy()
df_type1_upper = df_type1.apply({lambda x:x.upper()})
df_type1_upper

# c.求每个妖怪六项能力的离差，即所有能力中偏离中位数最大的值，添加到 df 并从大到小排序 

# 我的方法：
# 先求各项能力的中位数，再求各项能力与对应中位数的差的绝对值的最大值

df['diff'] = (df[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']] - df[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']].median()).abs().max(axis=1)
df.sort_values(by='diff',ascending=False).head()

# 中位数结果：
# df[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']].median()
# HP         65.0
# Attack     75.0
# Defense    70.0
# Sp. Atk    65.0
# Sp. Def    70.0
# Speed      65.0
# dtype: float64

20-65,10-75,230-70,10-65,230-70,5-65

# 答案是 各项能力-各项能力所有的中位数
# 但实际这样写是错误的，因为x.median()是所有数据的中位数，而不是每个数据的中位数
# df['Deviation'] = df[['HP', 'Attack', 'Defense', 'Sp. Atk',
#                      'Sp. Def', 'Speed']].apply(lambda x:np.max(
#                      (x-x.median()).abs()), 1)
# df.sort_values(by='Deviation',ascending=False).head()

# 修改版本：
# 1. 定义自定义函数：计算单条数据（单个妖怪）的最大偏离中位数
def calculate_max_deviation(abilities):
    """
    输入：单个妖怪的六项能力（Series，如HP、Attack等）
    输出：该妖怪能力偏离自身中位数的最大绝对值（离差）
    """
    # 步骤1：计算该妖怪自身六项能力的中位数
    ability_median = abilities.median()
    # 步骤2：计算每项能力与中位数的偏差（保留正负）
    deviations = abilities - ability_median
    # 步骤3：取偏差的绝对值，再找最大值（即“偏离中位数最大的值”）
    max_abs_deviation = deviations.abs().max()
    return max_abs_deviation

# 2. 应用函数到DataFrame，生成Deviation列（逻辑与原代码完全一致）
df['Deviation'] = df[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].apply(
    calculate_max_deviation,  # 调用自定义函数
    axis=1  # axis=1表示“按行应用”（每个行对应一个妖怪）
)

0    low
1    mid
2    mid
3    mid
4    mid
Name: Attack, dtype: object


### Ex2：指数加权窗口

#### 1. 作为扩张窗口的 ewm 窗口
在扩张窗口中，用户可以使用各类函数进行历史的累计指标统计，但这些内置的统计函数往往把窗口中的所有元素赋予了同样的权重。事实上，可以给出不同的权重来赋给窗口中的元素，指数加权窗口就是这样一种特殊的扩张窗口。

其中，最重要的参数是 `alpha`，它决定了默认情况下的窗口权重为：  
$ w_i = (1 - \alpha)^i, i \in \{0, 1, \dots, t\} $  
其中 $ i = t $ 表示当前元素，$ i = 0 $ 表示序列的第一个元素。  

从权重公式可以看出，离开当前值越远则权重越小，若记原序列为 $ x $，更新后的当前元素为 $ y_t $，此时通过加权公式归一化后可知：  
$
\begin{align}
y_t &= \frac{\sum_{i=0}^t w_i x_{t-i}}{\sum_{i=0}^t w_i} \\
&= \frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + \dots + (1 - \alpha)^t x_0}{1 + (1 - \alpha) + (1 - \alpha)^2 + \dots + (1 - \alpha)^t}
\end{align}
$

对于 `Series` 而言，可以用 `ewm` 对象如下计算指数平滑后的序列：  

```python
np.random.seed(0)

s = pd.Series(np.random.randint(-1,2,30).cumsum())

s.head()
# 0   -1
# 1   -1
# 2   -2
# 3   -2
# 4   -2
# dtype: int32
s.ewm(alpha=0.2).mean().head()
# 0   -1.000000
# 1   -1.000000
# 2   -1.409836
# 3   -1.609756
# 4   -1.725845
# dtype: float64
```
请用 expanding 窗口实现。
#### 2.作为滑动窗口的 ewm 窗口

从第1问中可以看到， ewm 作为一种扩张窗口的特例，只能从序列的第一个元素开始加权。现在希望给定一个限制窗口 n ，只对包含自身的最近的 n 个元素作为窗口进行滑动加权平滑。请根据滑窗函数，给出新的 
 与 
 的更新公式，并通过 rolling 窗口实现这一功能。

In [64]:
np.random.seed(0)

s = pd.Series(np.random.randint(-1,2,30).cumsum())

s.head()
# 0   -1
# 1   -1
# 2   -2
# 3   -2
# 4   -2
# dtype: int32
s.ewm(alpha=0.2).mean().head()
# 0   -1.000000
# 1   -1.000000
# 2   -1.409836
# 3   -1.609756
# 4   -1.725845
# dtype: float64

# 1、请用 expanding 窗口实现。
def my_ewm(x,alpha):
    # len(x.index.values) == len(x)
    # 指数部分使用了np数组
    # 
    # 第一个win结果 是 0.8**0 = 1
    # 第二个win结果 是 0.8**1 = 0.8 0.8**0 = 1 
    # 第三个win结果 是 0.8**2 = 0.64 0.8**1 = 0.8 0.8**0 = 1 
    # 以此类推

    win = (1-alpha)**np.arange(x.shape[0])[::-1]
    res = (x*win).sum() / win.sum()

    return res


s.expanding().apply(my_ewm,args=(0.2,)).head()
# 0   -1.000000
# 1   -1.000000
# 2   -1.409836
# 3   -1.609756
# 4   -1.725845
# dtype: float64

# 2.作为滑动窗口的 ewm 窗口

# 从第1问中可以看到， ewm 作为一种扩张窗口的特例，只能从序列的第一个元素开始加权。现在希望给定一个限制窗口 n ，只对包含自身的最近的 n 个元素作为窗口进行滑动加权平滑。请根据滑窗函数，给出新的 
#  与 
#  的更新公式，并通过 rolling 窗口实现这一功能。
s.rolling(4).apply(my_ewm,args=(0.2,)).head()

0         NaN
1         NaN
2         NaN
3   -1.609756
4   -1.826558
dtype: float64